# Data Information:

## Input variables:


## Output variable (desired target):


# Data preprocessing 

Import libraries and data, select a small sample set for trainning


In [69]:
from sklearn import datasets
import numpy as np
import pandas as pd

lol_origin = pd.read_csv('./leagueoflegends/leagueoflegends.csv')
lol = lol_origin

In [70]:
for x in ['golddiff','goldblueTop','goldblueJungle','goldblueMiddle','goldblueADC','goldblueSupport','goldredTop','goldredJungle','goldredMiddle','goldredADC','goldredSupport']:
    lol[x] = [int(y.split(',')[14]) for y in lol[x]]

for x in ['Top','Middle','ADC','Support']:
    lol['golddiff' + x ] = lol['goldblue' + x ] - lol['goldred' + x ]
    del lol['goldblue' + x ]
    del lol['goldred' + x ]


In [71]:
lol

,League,Year,Season,Type,blueTeamTag,bResult,rResult,redTeamTag,gamelength,golddiff,...,redADC,redADCChamp,redSupport,redSupportChamp,redBans,Address,golddiffTop,golddiffMiddle,golddiffADC,golddiffSupport
0,NALCS,2015,Spring,Season,TSM,1,0,C9,40,-790,...,Sneaky,Sivir,LemonNation,Thresh,"['Tristana', 'Leblanc', 'Nidalee']",http://matchhistory.na.leagueoflegends.com/en/...,-747,565,-98,-43
1,NALCS,2015,Spring,Season,CST,0,1,DIG,38,1394,...,CoreJJ,Corki,KiWiKiD,Annie,"['RekSai', 'Janna', 'Leblanc']",http://matchhistory.na.leagueoflegends.com/en/...,824,131,-192,117
2,NALCS,2015,Spring,Season,WFX,1,0,GV,40,2922,...,Cop,Corki,BunnyFuFuu,Janna,"['Leblanc', 'Zed', 'RekSai']",http://matchhistory.na.leagueoflegends.com/en/...,655,1283,317,612
3,NALCS,2015,Spring,Season,TIP,0,1,TL,41,-335,...,KEITH,KogMaw,Xpecial,Janna,"['RekSai', 'Rumble', 'LeeSin']",http://matchhistory.na.leagueoflegends.com/en/...,511,-42,312,-97
4,NALCS,2015,Spring,Season,CLG,1,0,T8,35,-574,...,Maplestreet8,Corki,Dodo8,Annie,"['Rumble', 'Sivir', 'Rengar']",http://matchhistory.na.leagueoflegends.com/en/...,-264,-164,474,37
5,NALCS,2015,Spring,Season,DIG,0,1,TIP,24,781,...,Apollo,Corki,Adrian,Thresh,"['Azir', 'RekSai', 'Lissandra']",http://matchhistory.na.leagueoflegends.com/en/...,-350,372,181,375
6,NALCS,2015,Spring,Season,CST,1,0,WFX,39,1017,...,Altec,Tristana,Gleeb,Nami,"['RekSai', 'Gnar', 'Kassadin']",http://matchhistory.na.leagueoflegends.com/en/...,632,189,-381,-151
7,NALCS,2015,Spring,Season,TL,1,0,CLG,43,-303,...,Doublelift,Sivir,Aphromoo,Blitzcrank,"['Nunu', 'Gnar', 'RekSai']",http://matchhistory.na.leagueoflegends.com/en/...,167,56,-611,184
8,NALCS,2015,Spring,Season,C9,0,1,GV,41,317,...,Cop,Corki,Bunny FuFuu,Morgana,"['Rumble', 'Lissandra', 'Fizz']",http://matchhistory.na.leagueoflegends.com/en/...,1334,-236,44,-523
9,NALCS,2015,Spring,Season,T8,1,0,TSM,32,17,...,WildTurtle,Graves,Lustboy,Morgana,"['RekSai', 'JarvanIV', 'Lissandra']",http://matchhistory.na.leagueoflegends.com/en/...,352,-374,-285,59


In [72]:
def get_data15(feature):
    lol15 = [x[2:-2] for x in feature]
    i=0
    data15 = []
    for y in lol15:
        y = y.split('], [')
        i = i + 1
        #print(type(y))
        #y = y.rsplit()
        #print(i)
        j = 0
        for x in y:
            if x.strip() =='':
                #print(bkills)
                #print(i)
                z = 0
                #bkills.append(z)
                #print(bkills)
                break
            else:
                x=x.split(',')
                z =float(x[0])
                if z < 15.0 :
                    j = j + 1
                    #print(j)
                    #print(x[0])
                    #print(j)
                else:

                    #print(bkills)
                    break
        data15.append(j)
        #print(bkills)
    #print(data15)
    #print(len(feature))
    #print(len(data15))
    return data15    
        
    
''' 
    for x in y:
        print(x[j])
        j= j+6
        
    for x in y:
        print(x[0])
    y = i
'''       

' \n    for x in y:\n        print(x[j])\n        j= j+6\n        \n    for x in y:\n        print(x[0])\n    y = i\n'

In [73]:
lol['bKills'] = get_data15(lol['bKills'])

In [74]:
lol['bKills']

0       1
1       5
2       8
3       2
4       1
5       3
6       5
7       2
8       2
9       2
10      0
11      3
12      2
13      3
14      3
15      2
16      0
17      1
18      2
19      0
20      4
21      2
22      3
23      2
24      1
25      1
26      1
27      3
28      5
29      4
       ..
7590    0
7591    1
7592    0
7593    5
7594    3
7595    1
7596    3
7597    3
7598    3
7599    4
7600    1
7601    0
7602    4
7603    3
7604    3
7605    1
7606    6
7607    0
7608    2
7609    1
7610    7
7611    2
7612    2
7613    2
7614    1
7615    4
7616    0
7617    2
7618    2
7619    3
Name: bKills, Length: 7620, dtype: int64

In [119]:
for y in lol15:
    print(y[0])

16.549
8.453
13.224
3.285
15.868
7.145
12.99
12.105
12.111
4.734
6.44
12.414
5.206
12.643
14.888
18.348
12.876
8.265
3.652
14.203
2.656
9.067
6.924
26.376
3.0
6.571
6.443
14.137
9.501
5.125
13.729
6.199
3.483
3.808
8.744
3.102
10.252
6.397
12.937
3.807
5.253
23.346
5.964
3.652
7.788
11.346
16.276
6.355
3.825
11.028
6.656
5.484
15.739
7.274
6.277
3.336
9.147
9.053
3.645
11.779
7.374

3.429
19.11
9.86
7.208
14.282
3.759
5.405
3.498
7.818
5.718
5.443
12.825
9.521
14.826
4.185
14.849
21.655
8.952
10.505
8.021
4.07
7.504
8.699
4.889
8.508
9.601
8.95
3.825
8.853
6.422
7.219
9.275
5.42
5.212
3.271
20.532
3.047
5.987
9.946
8.54
10.116
8.796
13.302
7.512
4.766
16.723
3.135
3.822
11.407
3.747
7.597
4.344
11.681
9.834
6.331
16.036
7.997
12.35
18.331
1.052
17.18
25.376
4.671
14.676
10.445
8.847
9.698
8.105
4.247
13.309
11.451
16.455
16.926
7.216
9.022
3.271
11.514
12.644
8.605
11.41
6.636
13.634
4.688
20.806
6.797
5.897
9.292
7.986
7.714
8.513
5.468
15.951
5.422
5.923
8.691
10.184
2.965
4.977
8.14

### Processing the ordinal and nominal data,seperately

In my opinion, only 'player' variable is ordinal


### Splitting data into 70% training and 30% test data 
### Bringing features onto the same scale

Here we choose standardization instead of normalization

Standardizing both the numeric and the Categorical features

<br>
<br>